# Latin Hyper Cube Sampling

In [1]:
import numpy as np
import pandas as pd
import math


In [2]:
def lhs(n_samples, bounds=None, seed=None):
    np.random.seed(seed)
    n_dim = len(bounds)
    result = np.empty((n_samples, n_dim), dtype=int)

    for i in range(n_dim):
        low, high = bounds[i]
        num_values = high - low + 1

        if num_values < n_samples:
            raise ValueError(
                f"Dimension {i}: Only {num_values} unique integer values available "
                f"for {n_samples} requested samples. Increase the range or reduce the sample size."
            )

        values = np.random.choice(np.arange(low, high + 1), size=n_samples, replace=False)
        np.random.shuffle(values)
        result[:, i] = values
    return result

**Parameter Ranges:**  
potential1 = [-2000, 2000]  
potential2 = [-2000, 2000]  
potential3 = 900  
hold1 = [0, 1800]  
hold2 = 0  
sweep_speed = [10, 1000]  
cycle = 100

In [3]:
bounds = [
    [-2000, 2000],    # potential1
    [-2000, 2000],    # potential2
    [0, 1800],        # hold1
    [10, 1000],       # sweep_speed
]


samples = lhs(n_samples=10, bounds=bounds, seed=42)
samples

array([[-1930,   214,   599,   712],
       [  990,  1451,   853,   320],
       [  554,  -855,   755,   406],
       [-1789,   488,  1439,   439],
       [-1445,   580,   134,   956],
       [  793,  -877,  1716,   293],
       [-1804,  -235,    54,   887],
       [-1197,  -338,  1227,    84],
       [-1473,  1025,   305,   857],
       [ 1871,   829,   377,   678]])

### Create Table

In [4]:
potential1 = samples[:, 0]
potential2 = samples[:, 1]
hold1 = samples[:, 2]
sweep_speed = samples[:, 3]

num_samples = len(potential1)
potential3 = [900] * num_samples
hold2 = [0] * num_samples
#cycles = [100] * num_samples

interval1 = hold1 + np.abs(potential1 - potential2)/sweep_speed
interval2 = np.abs(potential2 - potential3)/sweep_speed + hold2
cycles = [min(100, math.floor((3600 - interval1[i]) / interval2[i])) for i in range(len(interval1))]
duration = interval1 + cycles*interval2

sample_df = pd.DataFrame({
    'potential1 (mV vs Hg/HgO)': potential1,
    'hold1 (s)': hold1,
    'potential2 (mV vs Hg/HgO)': potential2,
    'hold2 (s)': hold2,
    'potential3 (mV vs Hg/HgO)': potential3,
    'sweep speed (mV/s)': sweep_speed,
    'cycle (P2-P3)': cycles,
    'duration (s)': duration,
    'duration (h)': duration/3600,
    'duration (min)': np.round(duration/60),
})

sample_df


,potential1 (mV vs Hg/HgO),hold1 (s),potential2 (mV vs Hg/HgO),hold2 (s),potential3 (mV vs Hg/HgO),sweep speed (mV/s),cycle (P2-P3),duration (s),duration (h),duration (min)
0,-1930,599,214,0,900,712,100,698.359551,0.193989,12.0
1,990,853,1451,0,900,320,100,1026.628125,0.285174,17.0
2,554,755,-855,0,900,406,100,1190.736453,0.330760,20.0
3,-1789,1439,488,0,900,439,100,1538.036446,0.427232,26.0
4,-1445,134,580,0,900,956,100,169.591004,0.047109,3.0
5,793,1716,-877,0,900,293,100,2328.184300,0.646718,39.0
6,-1804,54,-235,0,900,887,100,183.728298,0.051036,3.0
7,-1197,1227,-338,0,900,84,100,2711.035714,0.753065,45.0
8,-1473,305,1025,0,900,857,100,322.500583,0.089583,5.0
9,1871,377,829,0,900,678,100,389.008850,0.108058,6.0


### Test LHS

In [5]:
bounds = [
    [0, 10],
    [0, 10],
    [0, 10],
    [0, 10],
]

samples = lhs(n_samples=10, bounds=bounds, seed=42)
samples

array([[ 5,  2,  9, 10],
       [ 8,  5,  3,  0],
       [ 3,  9,  7,  5],
       [ 0,  0,  1,  4],
       [ 7,  6,  5,  1],
       [ 1,  3, 10,  2],
       [ 9, 10,  0,  9],
       [10,  7,  2,  8],
       [ 2,  8,  8,  7],
       [ 4,  4,  4,  3]])